# Web Scraping con BeautifulSoup4

In questo notebook vedremo come effettuare lo scraping di Immobiliare.it con **BeautifulSoup4**


## Import delle librerie
Per prima cosa importiamo le librerie necessarie:
- la libreria **requests** ci servirà per scaricare il contenuto della pagina
- la libreria *bs4* è BeautifulSoup4
- la libreria csv ci servirà per salvare i dati in formato CSV, oppure vederemo l'utilizzo di **pandas**
- tqdm ci fornisce una barra di avanzamento per verificare lo stato del caricamento
- json, è la libreria python per lavorare (e fare il parse) di un file in formato json

In [1]:
import requests
import bs4
import csv
from tqdm import tqdm_notebook as tqdm
import json
import pprint

## La libreria requests


**requests, use the HTTP protocol to exchange data over the web**

La libreria requests ci permette di ottenere il contenuto di una pagina web. In generale, dato un indirizzo web possiamo usare la libreria requests per fare il download di qualsiasi tipo di risorsa.

Impostiamo l'indirizzo URL da cui partire a scaricare (copiamo da Google Chrome l'indirizzo....).

In [2]:
num=1
webpage = f"https://www.immobiliare.it/vendita-case/milano-provincia/?criterio=rilevanza&pag={num}"

Attraverso f-strings (*formatted literal string*) riusciamo ad inizializzare velocemente l'indirizzo che ci serve. Partiremo dalla pagina *1* di www.immobiliare.it

In [3]:
print(webpage)

https://www.immobiliare.it/vendita-case/milano-provincia/?criterio=rilevanza&pag=1


Usiamo la **request** per scaricare il contenuto HTML della pagina web (simuliamo la chiamata del browset web).

Il metodo **get** effettua una richiesta di tipo GET come quella che facciamo quando navighiamo con un browser.
La risposta viene salvata nella variabile **response** che contiene:
- *response.status_code* lo stato della nostra chiama (se 200 tutto ok...)
- *response.content* la risposta in byte
- *response.text* il contenuto della risposta in formato testuale

In [ ]:
response = requests.get(webpage)
print("Status: " + str(response.status_code))
print("Text: " + str(response.text))


## BeautifulSoup
Passiamo il contenuto della pagina a **BS4**.

BS4 fa il parse del documento e restituisci il corrisponde albero. Possiamo usare diversi tipi di **parser**, quello di default è quello che ci serve per fare la maggior parte del lavoro: **html.parser**


Cosa è un parser?
Un parser riceve in input un file in formato strutturato (es. xml, json, ...) e restituisce un documenti navigabile attraverso API ad alto livello.


In [8]:
doc = bs4.BeautifulSoup(response.text)

Qualche funzione di base di BS4

doc.title
doc.head
doc.body

**Provate a complatare il codice.**
Che cosa restituiscono?

In [ ]:
doc.t...

In [ ]:
doc.h...

In [ ]:
doc.b...

Cerchiamo tutti gli annunci nella pagina.
Possiamo usare diversi metodi:
- **find**: restituisce UN solo elemento
- **find_all**: restituisce una lista di elementi
- **select**: posso specificare una query per ritornare tutti gli elementi che corrispondono

Per ognuno dei due metodi posso specificare delle condizioni:
- id="my_id" --> seleziona per id dell'elemento
- class_="my_class" --> seleziona per classe


Proviamo ad estrarre tutti i tag paragrafo **p** dalla pagina.

In [22]:
p = doc.find_all("p")
len(p)

65

Provate ora ad estrarre tutti i titoli **h1**... quanti sono? cosa contengono?

In [ ]:
h1 = doc.....
print(len(...))
print(h1[0].text)

Cerchiamo tutti gli annunci nella pagina.

In [14]:
annunci = doc.find(id="listing-container").find_all('li',class_="listing-item--wide")

In [15]:
len(annunci)

25

25 Annunci !!! Perfetto!


Andiamo ad esportare i dati degli annunci delle case in vendita.

**Provate a completare il codice per estrarre tutti gli attributi:**


In [ ]:
annunci_list = []
for annuncio in annunci:
    titolo = annuncio.find("p",class_="titolo").text
    #
    # estraiamo il campo data-id che rappresenta l'id dell'annuncio
    #
    #
    annuncio_id = annuncio["data-id"]
    url = annuncio.find("p",class_="titolo").find("a")["href"]
    prezzo = annuncio.find(...,...).text
    descrizione_breve = annuncio.find("div",class_="descrizione").text
    #
    # appendiamo il record alla nostra lista di annunci
    #
    annunci_list.append({'id': annuncio_id, 'titolo': titolo, 'link': url, 
                         'prezzo': prezzo, 'descrizione_breve': descrizione_breve})

len(annunci_list)

Ora andiamo a scaricare tutti gli annunci di Immobiliare.it relativi alla provincia di Milano.
Dobbiamo *ciclare* sulle pagine finché non troviamo più annunci...



Primo passo cerchiamo di scaricare 10 pagine...

**Completate la funzione in base all'esempio singolo**

In [16]:
def parse_annuncio(annuncio):
    titolo = annuncio.find("p",class_="titolo").text
    #
    # estraiamo il campo data-id che rappresenta l'id dell'annuncio
    #
    #
    annuncio_id = annuncio["data-id"]
    url = annuncio.find("p",class_="titolo").find("a")["href"]
    prezzo = annuncio.find("li",class_="lif__pricing").text
    descrizione_breve = annuncio.find("div",class_="descrizione").text
    #
    # ritorniamo il record
    #
    return {'id': annuncio_id, 'titolo': titolo, 'link': url, 
                         'prezzo': prezzo, 'descrizione_breve': descrizione_breve}


**A questo punto facciamo un ciclo fino alla pagina 10... provate a completarlo!**

In [17]:
annunci_list = []
for num in tqdm(range(1,11)):
    webpage = f"https://www.immobiliare.it/vendita-case/milano-provincia/?criterio=rilevanza&pag={num}"
    response = requests.get(webpage)
    doc = bs4.BeautifulSoup(response.text)
    annunci = doc.find(id="listing-container").find_all('li',class_="listing-item--wide")
    for annuncio in annunci:
        annunci_list.append(parse_annuncio(annuncio))

print(len(annunci_list))
        


250


Avete visto la libreria **tqdm**!
Trovate a questo link tutta la documentazione **https://github.com/tqdm/tqdm**:
è molto utile per rendere più piacevole il nostro notebook

## Come terminare lo scraping?

Problema: *quando mi fermo?*
- Posso cercare all'interno della pagina il numero di annuncio e dividerlo per 25
- Posso controllare lo stato della request (se != 200...)
- Posso controllare il numero di annunci trovati (se > 0....)
- Altre tecniche più avanzate

Per questo esercizio scegliamo di fermarci a 100 pagine...

In [ ]:
annunci_list = []
for num in tqdm(range(1,101)):
    webpage = f"https://www.immobiliare.it/vendita-case/milano-provincia/?criterio=rilevanza&pag={num}"
    response = requests.get(webpage)
    doc = bs4.BeautifulSoup(response.text)
    annunci = doc.find(id="listing-container").find_all('li',class_="listing-item--wide")
    for annuncio in annunci:
        try:
            annunci_list.append(parse_annuncio(annuncio))
        except:
            pass

print(len(annunci_list))

## Pandas

**Bene!** Iniziamo a vedere un'altra libreria fondamentale per lavorare con i dati!
Si tratta di **pandas**: *Python library for data analisys*
In pratica con pandas possiamo manipolare un set di dati o una serie storica in Python.

Per ora iniziamo solo a dare un paio di concetti.

Alla base di Pandas ci sono due tipi di dati: **Series** ed **DataFrame**:
- Series rappresenta una lista di dati
- DataFrame rappresentano un set di dati in formato tabellare

Ogni colonna di un DataFrame è una Series.

Possiamo crare un **DataFrame** utilizzato il metodo *pd.DataFrame* passando in input come parametro il nostro dizionario.

Un DataFrame pandas può essere facilmente esportato in formato CSV, Excel, ...


In [ ]:
import pandas as pd
ds_annunci = pd.DataFrame(annunci_list)
ds_annunci.set_index("id")
ds_annunci.head()

Il metodo **.info()** fornisce indicazione sulla struttura e sui dati del DataFrame.

In [ ]:
ds_annunci.info()

In [23]:
ds_annunci.to_csv("/home/master/immobiliare_annunci.csv")

## Le pagine degli annunci ###
Ora l'obiettivo è scaricare i dettagli dei singoli annunci e le immagini delle case.

**Pandas** fornisce il metodo *.read_csv* che permette di caricare in file in formato CSV all'interno di un DataFrame.


In [5]:
# aprimo il file csv
import pandas as pd
ds_annunci = pd.read_csv("/home/master/dataset/immobiliare/immobiliare_annunci.csv", index_col="id")
ds_annunci.head()

,Unnamed: 0,descrizione_breve,link,prezzo,titolo
id,,,,,
60469012,0,"\nSplendido 2 locali ultimo piano, centro stor...",https://www.immobiliare.it/60469012-Vendita-Bi...,\n € 107.000\n,\n\n Bilocale via Moden...
72525508,1,"\nSegrate, tre locali\nSegrate, in posizione c...",https://www.immobiliare.it/72525508-Vendita-Tr...,\n € 269.000\n,\n\n Trilocale via 1 Ma...
72531510,2,"\nSegrate, due locali\nSegrate, ideale per gio...",https://www.immobiliare.it/72531510-Vendita-Bi...,\n € 116.000\n,"\n\n Bilocale via Roma,..."
72717754,3,\nAdiacenze p.zza piola vendesi splendido appa...,https://www.immobiliare.it/72717754-Vendita-Ap...,\n € 980.000\n,\n\n Appartamento via L...
72673202,4,\nAd.ze piazzale bologna vendesi trilocale da ...,https://www.immobiliare.it/72673202-Vendita-Tr...,\n € 185.000\n,\n\n Trilocale via Dome...


**Per ogni annuncio andiamo a richiamre il link... come facciamo secondo voi?**
Facciamo una prova con i primi 5 annunci.


In [ ]:
dettagli = []
for annuncio_id, annuncio in ds_annunci.head().iterrows():
    link = annuncio["link"]
    response = requests.get(link)
    doc = bs4.BeautifulSoup(response.text)
    descrizione = doc.select(...)[0].text
    caratteristiche = doc.select("div.im-property__features ul.list-piped > li")
    indirizzo = doc.select(.....).....
    locali = caratteristiche[0].text
    superficie = caratteristiche[1].text
    bagni = caratteristiche[2].text
    piano = caratteristiche[3].text
    garanzia = caratteristiche[4].text
    dettagli.append({'id': annuncio_id, 'descrizione': ..., 'locali': ...,
                     'superficie': ..., 'bagni': ...,
                     'piano': ..., 'garanzia': ..., 'indirizzo': indirizzo})
    # scarichiamo le immagini
    imgs = doc.select("div.showcase__item > img")
    i = 0
    for img in imgs:
        src = img['src']
        img_file = requests.get(src, stream=True)
        if img_file.status_code == 200:
            with open("/home/master/dataset/immobiliare/immagini/img_" + str(annuncio_id) + "_" + str(i) + ".jpg", 'wb') as f:
                f.write(img_file.content)
            i = i+1
len(dettagli)

In [ ]:
import pandas as pd
ds_dettagli = pd.DataFrame(dettagli)
ds_dettagli.set_index("id")
ds_dettagli.head()

Ok procediamo cercando di fare qualcosa di più strutturato....

In [35]:
ds_annunci.head()

,Unnamed: 0,descrizione_breve,link,prezzo,titolo
id,,,,,
60469012,0,"\nSplendido 2 locali ultimo piano, centro stor...",https://www.immobiliare.it/60469012-Vendita-Bi...,\n € 107.000\n,\n\n Bilocale via Moden...
72525508,1,"\nSegrate, tre locali\nSegrate, in posizione c...",https://www.immobiliare.it/72525508-Vendita-Tr...,\n € 269.000\n,\n\n Trilocale via 1 Ma...
72531510,2,"\nSegrate, due locali\nSegrate, ideale per gio...",https://www.immobiliare.it/72531510-Vendita-Bi...,\n € 116.000\n,"\n\n Bilocale via Roma,..."
72717754,3,\nAdiacenze p.zza piola vendesi splendido appa...,https://www.immobiliare.it/72717754-Vendita-Ap...,\n € 980.000\n,\n\n Appartamento via L...
72673202,4,\nAd.ze piazzale bologna vendesi trilocale da ...,https://www.immobiliare.it/72673202-Vendita-Tr...,\n € 185.000\n,\n\n Trilocale via Dome...


In [ ]:
dettagli = []
for annuncio_id, annuncio in tqdm(ds_annunci.head().iterrows(), total=ds_annunci.head().shape[0]):
    try:
        link = annuncio["link"]
        response = requests.get(link)
        doc = bs4.BeautifulSoup(response.text)
        descrizione = doc.select("div#description")[0].text
        caratteristiche = doc.select("div.im-property__features ul.list-piped > li .text-bold")
        indirizzo = doc.select("span.im-address__content")[0].text
        ## dati
        dati = doc.select("dd.col-xs-12.col-sm-8 > span")
        tipologia = ""
        if(len(dati)>0):
            tipologia=dati[0].text
        locali = ""
        superficie = ""
        bagni = ""
        piano = ""
        garanzia = ""
        try:
            locali = caratteristiche[0].text if(len(caratteristiche)>=1) else ""
            superficie = caratteristiche[1].text if(len(caratteristiche)>=2) else ""
            bagni = caratteristiche[2].text if(len(caratteristiche)>=3) else ""
            piano = caratteristiche[3].text if(len(caratteristiche)>=4) else ""
            garanzia = caratteristiche[4].text if(len(caratteristiche)>=5) else ""
        except:
            pass
        
        ## caratteristiche
        fibra_ottica = "N"
        porta_blindata = "N"
        arredato = "N"
        giardiano = "N"
        tags = doc.select("span.label-gray")
        for tag in tags:
            if tag.text == "Fibra ottica": fibra_ottica = "Y"
            if tag.text == "Porta blindata": porta_blindata = "Y"
            if tag.text == "Giardino comune": giardino = "Y"
            if tag.text == "Arredato": arredato = "Y"
        
        dettagli.append({'id': annuncio_id, 'descrizione': descrizione, 'locali': locali,
                         'superficie': superficie, 'bagni': bagni,
                         'piano': piano, 'garanzia': garanzia, 'indirizzo': indirizzo,
                         'tipologia': tipologia, 'link': link, 'fibra_ottica': fibra_ottica,
                        "arredato": arredato, 'porta_blindata': porta_blindata})
        
        imgs = doc.select("div.showcase__item > img")
        i = 0
        for img in imgs:
            src = img['src']
            img_file = requests.get(src, stream=True)
            if img_file.status_code == 200:
                with open("/home/master/dataset/immobiliare/immagini/img_" + str(annuncio_id) + "_" + str(i) + ".jpg", 'wb') as f:
                    f.write(img_file.content)
                i = i+1
    except:
        pass


len(dettagli)

In [ ]:
import pandas as pd
ds_dettagli = pd.DataFrame(dettagli)
ds_dettagli.set_index("id")
ds_dettagli.head()

In [ ]:
ds_dettagli.info()

In [ ]:
## per evitare di vedere dati troncati
#pd.set_option('display.max_colwidth', -1)
print(ds_dettagli[["link"]])

In [ ]:
ds_dettagli.to_csv("/home/master/dataset/immobiliare/immobiliare_dettagli.csv")

# API

Vediamo come utilizzare la libreria **requests** per agganciare API messe a disposizione dai nostri fornitori o dai nostri colleghi.

## Crarifai
Proviamo i servizi messi a disposizione da Clarifai per aumentare i dati che abbiamo a disposizione.

In [24]:
from clarifai.rest import ClarifaiApp

# impostate la vostra chiave!!!!
clarifai_key = "41a0bb727c0c462d8cd09aa359fa9649"
app = ClarifaiApp(api_key=clarifai_key)

# usiamo il modello generale
model = app.public_models.general_model

Utilizziamo il metodo **predict_by_filename** per ottenere i **concetti** all'interno della pagina.

In [ ]:

response = model.predict_by_filename("/home/master/dataset/immobiliare_orig/immagini/img_73748534_0.jpg")

Controlliamo il contenuto della risposta. Ci interessa trovare tutti i concenti contenuti e il loro score.

In [8]:
print(response)

{'status': {'code': 10000, 'description': 'Ok', 'req_id': '89465528daa645e2abb73d985762be49'}, 'outputs': [{'id': '5aaf84c1be8f4b7cb1dc40a692133e62', 'status': {'code': 10000, 'description': 'Ok'}, 'created_at': '2019-05-22T10:02:33.499186393Z', 'model': {'id': 'aaa03c23b3724a16a56b629203edc62c', 'name': 'general', 'created_at': '2016-03-09T17:11:39.608845Z', 'app_id': 'main', 'output_info': {'message': 'Show output_info with: GET /models/{model_id}/output_info', 'type': 'concept', 'type_ext': 'concept'}, 'model_version': {'id': 'aa9ca48295b37401f8af92ad1af0d91d', 'created_at': '2016-07-13T01:19:12.147644Z', 'status': {'code': 21100, 'description': 'Model trained successfully'}}, 'display_name': 'General'}, 'input': {'id': '95e397413b5f4b49ae961824e5f5f9ca', 'data': {'image': {'url': 'https://s3.amazonaws.com/clarifai-api/img3/prod/orig/00eb67caa5984d29a2a4b953dee98649/4b2de98cfe2e90b3efaff347a0587eea', 'base64': 'dHJ1ZQ=='}}}, 'data': {'concepts': [{'id': 'ai_c9n7SB25', 'name': 'furni

Recuperiamo i dati...

In [6]:
if(response['status']['description'] == "Ok"):
    for concept in response["outputs"][0]["data"]["concepts"]:
        name = concept["name"]
        value = concept["value"]
        print(name + " " + str(value))

furniture 0.9990891218185425
room 0.9975548386573792
indoors 0.9934395551681519
interior design 0.9901063442230225
seat 0.988335371017456
window 0.980469822883606
bedroom 0.9745944738388062
lamp 0.973411500453949
home 0.9732177257537842
table 0.9710896611213684
chair 0.9704452157020569
house 0.9665247797966003
bed 0.9629145264625549
family 0.9598959684371948
sofa 0.9572453498840332
rug 0.9556664228439331
cushion 0.9493708610534668
curtain 0.9439741969108582
trading floor 0.9394346475601196
pillow 0.9329138994216919


# Geocoding

Cosa è il geocoding?

Localizziamo ora la casa in vendita con le API di **MapQuest**



In [31]:
indirizzo = "via finazzi 46, carvico(bg)"
key = "r3xgNwBAviBMdOj4Op90kvyy2iO1CDGz"
geocode_url = f"http://www.mapquestapi.com/geocoding/v1/address?key={key}&location={indirizzo}"
response = requests.get(geocode_url)

In [34]:
import json
geo = json.loads(response.text)
print(response.text)

{"info":{"statuscode":0,"copyright":{"text":"\u00A9 2019 MapQuest, Inc.","imageUrl":"http://api.mqcdn.com/res/mqlogo.gif","imageAltText":"\u00A9 2019 MapQuest, Inc."},"messages":[]},"options":{"maxResults":-1,"thumbMaps":true,"ignoreLatLngInput":false},"results":[{"providedLocation":{"location":"via finazzi 46, carvico(bg)"},"locations":[{"street":"","adminArea6":"","adminArea6Type":"Neighborhood","adminArea5":"Carvico","adminArea5Type":"City","adminArea4":"BG","adminArea4Type":"County","adminArea3":"Lombardy","adminArea3Type":"State","adminArea1":"IT","adminArea1Type":"Country","postalCode":"","geocodeQualityCode":"A5XAX","geocodeQuality":"CITY","dragPoint":false,"sideOfStreet":"N","linkId":"282493759","unknownInput":"","type":"s","latLng":{"lat":45.703779,"lng":9.482249},"displayLatLng":{"lat":45.703779,"lng":9.482249},"mapUrl":"http://www.mapquestapi.com/staticmap/v5/map?key=r3xgNwBAviBMdOj4Op90kvyy2iO1CDGz&type=map&size=225,160&locations=45.703779,9.482249|marker-sm-50318A-1&scaleb

In [40]:
print(geo['results'][0]['locations'][0]['latLng']['lat'])
print(geo['results'][0]['locations'][0]['latLng']['lng'])

45.703779
9.482249


## Scraping completo da Immobiliare.it

In [11]:
ds_annunci.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2460 entries, 60469012 to 73303904
Data columns (total 5 columns):
Unnamed: 0           2460 non-null int64
descrizione_breve    2460 non-null object
link                 2460 non-null object
prezzo               2460 non-null object
titolo               2460 non-null object
dtypes: int64(1), object(4)
memory usage: 115.3+ KB


Aggiungiamo al nostro codice la chiamata a Clarify. Abbiamo solo 5.000 chiamate,
quindi ci limitiamo alla prima immagine.

In [ ]:
dettagli = []
immagini = []
for annuncio_id, annuncio in tqdm(ds_annunci.iterrows(), total=ds_annunci.shape[0]):
    try:
        link = annuncio["link"]
        response = requests.get(link)
        doc = bs4.BeautifulSoup(response.text)
        descrizione = doc.select("div#description")[0].text
        caratteristiche = doc.select("div.im-property__features ul.list-piped > li .text-bold")
        indirizzo = doc.select("span.im-address__content")[0].text
        ## dati
        dati = doc.select("dd.col-xs-12.col-sm-8 > span")
        tipologia = ""
        if(len(dati)>0):
            tipologia=dati[0].text
        locali = ""
        superficie = ""
        bagni = ""
        piano = ""
        garanzia = ""
        try:
            locali = caratteristiche[0].text if(len(caratteristiche)>=1) else ""
            superficie = caratteristiche[1].text if(len(caratteristiche)>=2) else ""
            bagni = caratteristiche[2].text if(len(caratteristiche)>=3) else ""
            piano = caratteristiche[3].text if(len(caratteristiche)>=4) else ""
            garanzia = caratteristiche[4].text if(len(caratteristiche)>=5) else ""
        except:
            pass
        
        ## caratteristiche
        fibra_ottica = "N"
        porta_blindata = "N"
        arredato = "N"
        giardiano = "N"
        tags = doc.select("span.label-gray")
        for tag in tags:
            if tag.text == "Fibra ottica": fibra_ottica = "Y"
            if tag.text == "Porta blindata": porta_blindata = "Y"
            if tag.text == "Giardino comune": giardino = "Y"
            if tag.text == "Arredato": arredato = "Y"
        
        ## geocoding
        lat = ""
        lon = ""
        try:
            key = "r3xgNwBAviBMdOj4Op90kvyy2iO1CDGz"
            geocode_url = f"http://www.mapquestapi.com/geocoding/v1/address?key={key}&location={indirizzo}"
            response = requests.get(geocode_url)
            geo = json.loads(response.text)
            lat = geo['results'][0]['locations'][0]['latLng']['lat']
            lon = geo['results'][0]['locations'][0]['latLng']['lon']
        except:
            pass
        
        dettagli.append({'id': annuncio_id, 'descrizione': descrizione, 'locali': locali,
                         'superficie': superficie, 'bagni': bagni,
                         'piano': piano, 'garanzia': garanzia, 'indirizzo': indirizzo,
                         'tipologia': tipologia, 'link': link, 'fibra_ottica': fibra_ottica,
                        "arredato": arredato, 'porta_blindata': porta_blindata, 'lat': lat, 'lon': lon})
        
        ## immagini
        imgs = doc.select("div.showcase__item > img")
        i = 0
        for img in imgs:
            src = img['src']
            img_file = requests.get(src, stream=True)
            if img_file.status_code == 200:
                with open("/home/master/dataset/immobiliare/immagini/img_" + str(annuncio_id) + "_" + str(i) + ".jpg", 'wb') as f:
                    f.write(img_file.content)
                i = i+1
            if(i==1):
                filename = "/home/master/dataset/immobiliare/immagini/img_" + str(annuncio_id) + "_0.jpg"
                response = model.predict_by_filename(filename) 
                if(response['status']['description'] == "Ok"):
                    for concept in response["outputs"][0]["data"]["concepts"]:
                        name = concept["name"]
                        value = concept["value"]
                        immagini.append({'id': annuncio_id, 'immagine': filename, 'name': name,
                         'value': value, 'src': src})
    except:
        pass


len(dettagli)

In [15]:
import pandas as pd
ds_dettagli = pd.DataFrame(dettagli)
ds_dettagli.set_index("id")
ds_dettagli.head()


ds_immagini = pd.DataFrame(immagini)
ds_immagini.head()

,id,immagine,name,src,value
0,72525508,/home/master/dataset/immobiliare/immagini/img_...,architecture,https://pic.im-cdn.it/image/777419290/m-c.jpg,0.995366
1,72525508,/home/master/dataset/immobiliare/immagini/img_...,building,https://pic.im-cdn.it/image/777419290/m-c.jpg,0.992247
2,72525508,/home/master/dataset/immobiliare/immagini/img_...,house,https://pic.im-cdn.it/image/777419290/m-c.jpg,0.991236
3,72525508,/home/master/dataset/immobiliare/immagini/img_...,home,https://pic.im-cdn.it/image/777419290/m-c.jpg,0.986825
4,72525508,/home/master/dataset/immobiliare/immagini/img_...,city,https://pic.im-cdn.it/image/777419290/m-c.jpg,0.967805


In [16]:
ds_dettagli.to_csv("/home/master/dataset/immobiliare/immobiliare_dettagli.csv")
ds_immagini.to_csv("/home/master/dataset/immobiliare/immobiliare_immagini.csv")

In [17]:
ds_immagini.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11840 entries, 0 to 11839
Data columns (total 5 columns):
id          11840 non-null int64
immagine    11840 non-null object
name        11840 non-null object
src         11840 non-null object
value       11840 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 462.6+ KB
